In [ ]:
import sys

sys.path.append('..')

from src.models.miniconvnext import MiniConvNeXt

import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
from pathlib import Path
import numpy as np

from src.trainer import Trainer
from src.dataset import HumanPosesDataset
from sklearn.model_selection import train_test_split
import torchvision.transforms as T
import torch

In [ ]:
import plotly.io as pio
pio.renderers.default = "browser"

# в прошлый раз моделька переобучилась быстро, в этом ноутбуке более сильныее аугментации

In [ ]:
from torchvision import transforms

mean = [0.4638, 0.4522, 0.4148]
std = [0.2222, 0.2198, 0.2176]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(size=224, scale=(0.5, 1.0), ratio=(0.75, 1.33)),
    transforms.RandomHorizontalFlip(p=0.7),
    transforms.RandomApply([
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2)
    ], p=0.5),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.5),
    transforms.Normalize(mean=mean, std=std),
])


val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])


In [ ]:
CSV_PATH = Path("../data/human_poses_data/train_answers.csv")
TRAIN_DIR = Path("../data/human_poses_data/img_train")

df = pd.read_csv(CSV_PATH)

train_ids, val_ids = train_test_split(
    df['img_id'].values,
    test_size=0.2,
    stratify=df['target_feature'],
    random_state=42
)

train_df = df[df['img_id'].isin(train_ids)].reset_index(drop=True)
val_df = df[df['img_id'].isin(val_ids)].reset_index(drop=True)

train_dataset = HumanPosesDataset(
    data_df=train_df,
    img_dir=TRAIN_DIR,
    transform=train_transform,
)

val_dataset = HumanPosesDataset(
    data_df=val_df,
    img_dir=TRAIN_DIR,
    transform=val_transform,
)



train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

In [ ]:
num_classes = len(np.unique(df['target_feature']))
print(f"Количество классов: {num_classes}")

# обучение

In [ ]:
model = MiniConvNeXt(num_classes=num_classes)

In [ ]:
NUM_EPOCH = 75

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=3e-4,
    steps_per_epoch=len(train_loader),
    epochs=NUM_EPOCH,
    pct_start=0.1,
    anneal_strategy='cos',
    div_factor=25.0,
    final_div_factor=1e4
)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
from src.utils import MixupCutMixAugmenter

mixup_cutmix_fn = MixupCutMixAugmenter(alpha=1.0, p_mixup=0.3)

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=NUM_EPOCH,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    batch_augment_fn=mixup_cutmix_fn,
    experiment_name="convnext_onecycle",
    use_wandb=True,
    seed=42,
)


history = trainer.train()

еще 75 сверху

In [ ]:
from src.utils import load_best_model

NUM_EPOCH = 75

model = load_best_model(model, "checkpoints/convnext_onecycle_best.pth", device=device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCH)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
from src.utils import MixupCutMixAugmenter

mixup_cutmix_fn = MixupCutMixAugmenter(alpha=1.0, p_mixup=0.5)

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=NUM_EPOCH,
    optimizer=optimizer,
    criterion=nn.CrossEntropyLoss(label_smoothing=0.1),
    scheduler=scheduler,
    batch_augment_fn=mixup_cutmix_fn,
    experiment_name="convnext_onecycle_retry",
    use_wandb=True,
    seed=42,
)


In [ ]:
history = trainer.train()

аваыъаыа еще

In [ ]:
from src.utils import load_best_model
import torch
import torch.nn as nn

NUM_EPOCH = 100

model = load_best_model(model, "checkpoints/convnext_onecycle_retry_best.pth", device=device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer, T_0=20, T_mult=2
)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
from src.utils import MixupCutMixAugmenter

mixup_cutmix_fn = MixupCutMixAugmenter(alpha=1.0, p_mixup=0.5)

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=NUM_EPOCH,
    optimizer=optimizer,
    criterion=nn.CrossEntropyLoss(label_smoothing=0.1),
    scheduler=scheduler,
    batch_augment_fn=mixup_cutmix_fn,
    experiment_name="convnext_onecycle_retry_last",
    use_wandb=True,
    seed=42,
)

In [ ]:
history = trainer.train()

предикт

In [ ]:
from src.utils import load_best_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = MiniConvNeXt(num_classes=num_classes)

model = load_best_model(model, "checkpoints/convnext_onecycle_retry_last_best.pth", device=device)

In [ ]:
import pandas as pd
from pathlib import Path

TEST_DIR = Path("data/human_poses_data/img_test")
test_ids = [f.stem for f in TEST_DIR.glob("*.jpg")]
test_df = pd.DataFrame({'img_id': test_ids})

In [ ]:
from pathlib import Path
import pandas as pd

TEST_DIR = Path("../data/human_poses_data/img_test")

test_ids = [f.stem for f in TEST_DIR.glob("*.jpg")]
test_df = pd.DataFrame({"img_id": test_ids})


test_dataset = HumanPosesDataset(
    data_df=test_df,
    img_dir=TEST_DIR,
    transform=val_transform,
    preload=False,
    mode='test',
)


test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


In [ ]:
from src.utils import make_submission

index_to_class = train_dataset.index_to_class

make_submission(
    model=model,
    test_loader=test_loader,
    device=device,
    index_to_class=train_dataset.index_to_class,
    output_path="submission.csv"
)


In [ ]:
!kaggle competitions submit -c ml-intensive-yandex-academy-spring-2025 -f submission.csv -m "Message"

0.63! поучу еще

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 128

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

In [ ]:
import torch
import torch.nn as nn

NUM_EPOCH = 50

model = MiniConvNeXt(num_classes=num_classes)

model = load_best_model(model, "checkpoints/convnext_onecycle_retry_last_best.pth", device=device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=NUM_EPOCH,
    eta_min=1e-5
)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
from src.utils import MixupCutMixAugmenter

mixup_cutmix_fn = MixupCutMixAugmenter(alpha=1.0, p_mixup=0.5)

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=NUM_EPOCH,
    optimizer=optimizer,
    criterion=nn.CrossEntropyLoss(label_smoothing=0.1),
    scheduler=scheduler,
    batch_augment_fn=mixup_cutmix_fn,
    experiment_name="convnext_onecycle_retry_last_last",
    use_wandb=True,
    seed=42,
)

In [ ]:
history = trainer.train()

опять сабмит

In [ ]:
from src.utils import load_best_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = MiniConvNeXt(num_classes=num_classes)

model = load_best_model(model, "checkpoints/convnext_onecycle_retry_last_last_best.pth", device=device)

In [ ]:
from src.utils import make_submission

make_submission(
    model=model,
    test_loader=test_loader,
    device=device,
    index_to_class=train_dataset.index_to_class,
    output_path="submission.csv"
)

In [ ]:
!kaggle competitions submit -c ml-intensive-yandex-academy-spring-2025 -f submission.csv -m "Message"

сейчас я попробую в конце просто стабильно поучиться с 256 батч сайз без миксап и катмикс
upd: мгновенно переобучился, так что не буду отключать, просто понижу вероятность аугментаций

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 256

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

In [ ]:
import torch
import torch.nn as nn
from src.utils import load_best_model

NUM_EPOCH = 50

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = MiniConvNeXt(num_classes=num_classes)

model = load_best_model(model, "checkpoints/convnext_onecycle_retry_last_last_best.pth", device=device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=NUM_EPOCH,
    eta_min=1e-6
)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

In [ ]:
from src.utils import MixupCutMixAugmenter

mixup_cutmix_fn = MixupCutMixAugmenter(alpha=1.0, p_mixup=0.2)

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=NUM_EPOCH,
    optimizer=optimizer,
    criterion=nn.CrossEntropyLoss(label_smoothing=0.1),
    scheduler=scheduler,
    batch_augment_fn=mixup_cutmix_fn,
    experiment_name="convnext_onecycle_retry_last_last",
    use_wandb=True,
    seed=42,
)

In [ ]:
history = trainer.train()

сабмит

In [ ]:
from src.utils import load_best_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = MiniConvNeXt(num_classes=num_classes)

model = load_best_model(model, "checkpoints/convnext_onecycle_retry_last_last_best.pth", device=device)

In [ ]:
from src.utils import make_submission

make_submission(
    model=model,
    test_loader=test_loader,
    device=device,
    index_to_class=train_dataset.index_to_class,
    output_path="submission.csv"
)

In [ ]:
!kaggle competitions submit -c ml-intensive-yandex-academy-spring-2025 -f submission.csv -m "Message"